<a href="https://colab.research.google.com/github/seosztt/Deep_Learning_from_Scratch_2/blob/master/ch4/chapter4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!ls

sample_data


In [3]:
cd '/gdrive/MyDrive/Deep_Learning_from_Scratch/chapter4'

/gdrive/MyDrive/Deep_Learning_from_Scratch/chapter4


In [4]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        W_in = 0.01 * np,random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:,i])
        h += 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout += 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [5]:
import sys
sys.path.append('..')
import numpy as np
from common import config
config.GPU = True
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from datasets import ptb

window_size=5
hidden_size=100
batch_size = 100
max_epoch=10

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params={}
params['word_vecs'] = word_vecs.astype(np.floar16)
params['word_to_id'] = word_to_id
parpam['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

TypeError: ignored

In [7]:
!pip install cupy

  Using cached cupy-9.5.0.tar.gz (1.7 MB)
  Created wheel for cupy: filename=cupy-9.5.0-cp37-cp37m-linux_x86_64.whl size=93352170 sha256=c53bd659143b552c14426af4b9afb156b8bcf659a4e8dcbc3dcd5c5d659b29d6
  Stored in directory: /root/.cache/pip/wheels/03/cf/00/c29a65b64519d1c3952308947ccba531cefd04b3c3b50d69a0
Successfully built cupy
